# ADS 509 Sentiment Assignment

This notebook holds the Sentiment Assignment for Module 6 in ADS 509, Applied Text Mining. Work through this notebook, writing code and answering questions where required. 

In a previous assignment you put together Twitter data and lyrics data on two artists. In this assignment we apply sentiment analysis to those data sets. If, for some reason, you did not complete that previous assignment, data to use for this assignment can be found in the assignment materials section of Blackboard. 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [2]:
import os
import re
import emoji
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
from string import punctuation

from nltk.corpus import stopwords

sw = stopwords.words("english")

In [6]:
# Add any additional import statements you need here

from glob import glob

In [5]:
# change `data_location` to the location of the folder on your machine.
data_location = "."

# These subfolders should still work if you correctly stored the 
# data from the Module 1 assignment
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

positive_words_file = "positive-words.txt"
negative_words_file = "negative-words.txt"
tidy_text_file = "tidytext_sentiments.txt"

## Data Input

Now read in each of the corpora. For the lyrics data, it may be convenient to store the entire contents of the file to make it easier to inspect the titles individually, as you'll do in the last part of the assignment. In the solution, I stored the lyrics data in a dictionary with two dimensions of keys: artist and song. The value was the file contents. A Pandas data frame would work equally well. 

For the Twitter data, we only need the description field for this assignment. Feel free all the descriptions read it into a data structure. In the solution, I stored the descriptions as a dictionary of lists, with the key being the artist. 




In [7]:
# Read in the lyrics data

songbank = []
for artistName in os.listdir("lyrics"):
    artist_path = os.path.join('lyrics',artistName)
    for lyric_file in os.listdir(artist_path):
        song_path = os.path.join('lyrics',artistName,lyric_file)
        with open(song_path, 'r') as f:
            songName = lyric_file.replace('.txt','').split('_')[-1]
            lyrics = f.read()
            songbank.append({
                "song_name": songName,
                "artist": artistName,
                "lyrics": lyrics,
            })

# create pandas dataframe
lyrics_data = pd.DataFrame(songbank)
lyrics_data.head()


,song_name,artist,lyrics
0,breakitoff,rihanna,Break It Off\n\n\n\nBreaking it off and settin...
1,crazylittlethingcalledlove,rihanna,"Crazy Little Thing Called Love\n\n\n\nOoh, yea..."
2,demhaters,rihanna,Dem Haters\n\nI'm not concerned with people\nW...
3,finalgoodbye,rihanna,Final Goodbye\n\nI never should've waited so l...
4,hereigoagain,rihanna,Here I Go Again\n\nNa-na-na-na-na\nNa-na-na-na...


In [9]:
# Read in the twitter data

artists = ['rihanna', 'taylorswift13']
twitterDFS = {}
for artist in artists:
    filePath = os.path.join("twitter", f'{artist}_followers_data.txt')
    twitterDFS[artist] = pd.read_csv(filePath,delimiter='\t')['description'].dropna().tolist()

#twitterDFS

In [17]:
# Read in the positive and negative words and the
# tidytext sentiment. Store these so that the positive
# words are associated with a score of +1 and negative words
# are associated with a score of -1. You can use a dataframe or a 
# dictionary for this.

pos = pd.read_csv(positive_words_file, skiprows=35, header=None).assign(score=1).rename(columns={0:'word'})
neg = pd.read_csv(negative_words_file, skiprows=35, header=None).assign(score=-1).rename(columns={0:'word'})
tidy = pd.read_csv(tidy_text_file, sep='\t')
tidy['score'] = np.where(tidy['sentiment'] == 'negative', -1, 1)
tidy.head()

,word,sentiment,lexicon,score
0,abandon,negative,nrc,-1
1,abandoned,negative,nrc,-1
2,abandonment,negative,nrc,-1
3,abba,positive,nrc,1
4,abduction,negative,nrc,-1


In [18]:
scores = pd.concat((pos, neg, tidy[['word', 'score']]), ignore_index=True)
scores.head()

,word,score
0,a+,1
1,abound,1
2,abounds,1
3,abundance,1
4,abundant,1


## Sentiment Analysis on Songs

In this section, score the sentiment for all the songs for both artists in your data set. Score the sentiment by manually calculating the sentiment using the combined lexicons provided in this repository. 

After you have calculated these sentiments, answer the questions at the end of this section.


In [38]:
def clean_text(txt):
    txt = txt.lower()
    txt = txt.replace('\n', ' ')
    txt = re.sub('\s+', ' ', txt)
    txt = re.sub('[^a-z\s]','', txt)
    return txt

In [39]:
# your code here
lyrics_data.head()

,song_name,artist,lyrics
0,breakitoff,rihanna,Break It Off\n\n\n\nBreaking it off and settin...
1,crazylittlethingcalledlove,rihanna,"Crazy Little Thing Called Love\n\n\n\nOoh, yea..."
2,demhaters,rihanna,Dem Haters\n\nI'm not concerned with people\nW...
3,finalgoodbye,rihanna,Final Goodbye\n\nI never should've waited so l...
4,hereigoagain,rihanna,Here I Go Again\n\nNa-na-na-na-na\nNa-na-na-na...


In [40]:
lyrics_data['lyrics'].apply(clean_text)

0     break it off breaking it off and setting it of...
1     crazy little thing called love ooh yeah yeah y...
2     dem haters im not concerned with people who pr...
3     final goodbye i never shouldve waited so long ...
4     here i go again nanananana nanananana nananana...
5     if its lovin that you want i wanna let you kno...
6     kisses dont lie kisses dont no they dont never...
7     let me boy let me let me why dont you let me l...
8     music of the sun ohohohohohoh ohohohoh yeah li...
9     now i know took a chance rolled the dice on me...
10    pon de replay come mr dj song pon de replay co...
11    pon de replay remix woo di club is getting war...
12    rush start time no miss rihanna kardinal offis...
13    selfish girl hey ay yeah yeah ay yeah yeah yea...
14    sos lalala lalala lalalalala oh you know ive n...
15    that la la la here we go oh la la la la la yea...
16    the last time once was a time you and i made a...
17    theres a thug in my life gotta keep it on 

### Questions

Q: Overall, which artist has the higher average sentiment per song? 

A: <!-- Your answer here -->

---

Q: For your first artist, what songs have the highest and lowest sentiments? Print those songs to the screen.

A: <!-- Your answer here -->

---

Q: For your second artist, what songs have the highest and lowest sentiments? Print those songs to the screen.

A: <!-- Your answer here -->

---

Q: Plot the distributions of the sentiment scores for both artists. You can use `seaborn` to plot densities or plot histograms in matplotlib.




## Sentiment Analysis on Twitter Descriptions

In this section, define two sets of emojis you designate as positive and negative. Make sure to have at least 10 emojis per set. You can learn about the most popular emojis on Twitter at [the emojitracker](https://emojitracker.com/). 

Associate your positive emojis with a score of +1, negative with -1. Score the average sentiment of your two artists based on the Twitter descriptions of their followers. The average sentiment can just be the total score divided by number of followers. 

In [ ]:
# your code here

Q: What is the average sentiment of your two artists? 

A: <!-- Your answer here --> 

---

Q: Which positive emoji is the most popular for each artist? Which negative emoji? 

A: <!-- Your answer here --> 

